In [1]:
!pip install faiss-cpu groq PyPDF2 pypdf google-cloud-aiplatform>=1.38

In [2]:
!pip install langchain langchain_google_genai langchain_community langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 724.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.6/396.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.5/290.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [28]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.13.2
    Uninstalling tomlkit-0.13.2:
      Successfully uninstalled tomlkit-0.13.2


In [1]:
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from google.colab import userdata

In [2]:
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [14]:
llm=ChatGroq(groq_api_key=GROQ_API_KEY, temperature=0.1,
             model_name="Llama3-8b-8192")

prompt=ChatPromptTemplate.from_template(
"""
You are an ai assistant. Use the following conversation history and context to answer the question.
If you don't know the answer, just say that you don't know. Don't try to make up an answer.

Conversation history:
{chat_history}

Context:
{context}

Questions: {input}

"""
)

In [4]:
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001", google_api_key=GOOGLE_API_KEY)

loader = PyPDFLoader("/content/Forest-Carbon-SMPP-Brief-20201030.pdf")

docs = loader.load()

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)

final_documents = text_splitter.split_documents(docs)

vectors=FAISS.from_documents(final_documents, embeddings)

In [15]:
document_chain = create_stuff_documents_chain(llm,prompt)

retriever = vectors.as_retriever()

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

retrieval_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
)

In [16]:
response = retrieval_chain.invoke('explain about the project briefly in one paragraph')
print(response['answer'])

The project aims to protect and regenerate a peatland rainforest in Sumatra, which was once one of the most biodiverse regions in the country. The project is partnering with the Zoological Society of London to conduct a comprehensive biodiversity assessment and catalog of the flora and fauna, and is also working to reduce wildlife poaching by providing new livelihood opportunities. The project is also focused on restoring the peatlands to their natural state, which has high rehabilitation and emissions reduction potential, and is eligible to produce and certify emissions reductions under an "avoided planned deforestation" scenario.


In [17]:
response = retrieval_chain.invoke({'question':'where is the location of the project?'})
print(response['answer'])

The location of the Sumatra Merang Peatland project is in the Musi Banyuasin District in South Sumatra, Indonesia.


In [18]:
response = retrieval_chain.invoke({'question':'How many hectares is the project restoring?'})
print(response['answer'])

According to the provided context, the Sumatra Merang Peatland project is restoring and protecting 22,934 hectares of peatland forest.


In [19]:
response = retrieval_chain.invoke({'question':'can you explain how about forest monitoring?'})
print(response['answer'])

According to the context, the project plans to monitor the forest through a combination of methods, including:

1. Ground teams patrolling the project area to monitor the health of the landscape.
2. Using technologies such as drones and satellite-based Early Warning Systems to prevent fires and illegal land clearance.

These methods will help the project team to effectively monitor the forest and prevent any potential threats to the protected area.


In [20]:
response = retrieval_chain.invoke({'question':'can you explain what sustainable banking is?'})
print(response['answer'])

I don't know the answer to this question based on the provided context. The text appears to be about a specific project, the Sumatra Merang Peatland project, and its goals and activities related to climate finance, community development, and peatland rehabilitation. It does not provide information about sustainable banking. If you're looking for information on sustainable banking, I'd be happy to help you find a reliable source or provide a general definition.


In [22]:
response = retrieval_chain.invoke({'question':'Why is the Sumatra Merang Peatland project so important to the surrounding local communities?'})
print(response['answer'])

According to the provided context, local communities play a significant role in the Sumatra Merang Peatland project. The project strengthens fire management and forest patrol through regular interaction with local community members, law enforcement, and Ministry of Forestry officials. The project's efforts to reduce logging and rehabilitate peatlands have been received positively by local communities, as this improves the health of the landscape.

The project also benefits local communities in several ways. For instance, community members, law enforcement, and project staff have worked together to destroy large machinery being used for illegal logging operations in the project area. Additionally, the project aims to create jobs and improve livelihoods, with 25% of jobs created being held by women. The project also supports local enterprises, such as sustainable fish production, and engages local people through community-led development and direct employment of 145 residents from the lo

In [24]:
response = retrieval_chain.invoke({'question':'What are the main challenges faced by local communities, and how does the project aim to address them?'})
print(response['answer'])

According to the provided context, the main challenges faced by local communities in the Sumatra Merang Peatland project area are:

1. Public health: The communities are situated directly on the river and have no access to clean water or sanitation infrastructure, making them highly susceptible to waterborne diseases.
2. Water/sanitation: The lack of access to clean water and sanitation infrastructure is a core challenge for local villages.

The project aims to address these challenges through the following initiatives:

1. Public health campaigns: The project will support public health campaigns to address the health concerns of local communities.
2. Water/sanitation infrastructure: The project will invest funds to improve public health by developing sustainable water and sanitation infrastructure in the local villages.
3. Education programs: The project will support education programs to raise awareness about the importance of public health and sanitation.

Additionally, the project 

In [25]:
response = retrieval_chain.invoke({'question':'What makes the project eligible to produce and certify emissions reductions under an "avoided planned deforestation" scenario?'})
print(response['answer'])

According to the provided context, the Sumatra Merang Peatland project is eligible to produce and certify emissions reductions under an "avoided planned deforestation" scenario because the surrounding land-use context suggests that the project area would be converted for paper pulp production if not for the project.


In [26]:
response = retrieval_chain.invoke({'question':'What environmental consequences could result from converting the project area to paper pulp production?'})
print(response['answer'])

According to the provided context, if the project area is converted for paper pulp production, it could result in the following environmental consequences:

* Rapid peat decomposition, which would dramatically increase greenhouse gas emissions.
* The risk of fire would increase due to the conversion of the area.
* The original vegetation would be completely cleared, and the area would be planted with trees commonly used for pulp and paper production on tropical peatlands.
* An extensive drainage system would be built, which would further accelerate peat decomposition and increase greenhouse gas emissions.

These consequences would likely have significant negative impacts on the environment, including increased greenhouse gas emissions, increased risk of fire, and loss of biodiversity.


In [32]:
import gradio as gr

def chatbot(query):
    response = retrieval_chain.invoke({'question':query})
    answer = response['answer']
    return answer

iface = gr.Interface(fn=chatbot, inputs="text", outputs="text", title="Ask Your PDF Document")

iface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://844fded7fc7aab0cfd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://844fded7fc7aab0cfd.gradio.live


In [31]:
print("Chat History: ", memory.load_memory_variables({}))

Chat History:  {'chat_history': [HumanMessage(content='explain about the project briefly in one paragraph'), AIMessage(content='The project aims to protect and regenerate a peatland rainforest in Sumatra, which was once one of the most biodiverse regions in the country. The project is partnering with the Zoological Society of London to conduct a comprehensive biodiversity assessment and catalog of the flora and fauna, and is also working to reduce wildlife poaching by providing new livelihood opportunities. The project is also focused on restoring the peatlands to their natural state, which has high rehabilitation and emissions reduction potential, and is eligible to produce and certify emissions reductions under an "avoided planned deforestation" scenario.'), HumanMessage(content='where is the location of the project?'), AIMessage(content='The location of the Sumatra Merang Peatland project is in the Musi Banyuasin District in South Sumatra, Indonesia.'), HumanMessage(content='How man